In [17]:
#start here this initializes the required modules

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase.io import read
from ase.build import molecule
from ase.io import gamess_us
from ase.io.gamess_us import read_gamess_us_out
from ase.visualize import view
from ase.build import molecule
from weas_widget import WeasWidget
from ase.calculators.gamess_us import GAMESSUS
from ase.io import write
import re

#now determine how many cores we have
ncor = ! nproc

slist_string = " ".join(ncor)
try:
    integer_value = int(slist_string)
    print("Converted value:", integer_value)
except ValueError:
    print("Unable to convert to integer:", slist_string)

ncor=integer_value

def clean_for_new_job(name):

    # clean up output name in current folder
    nameout= name + '.out'
    if os.path.exists(nameout):
        # If the file exists, delete it
        os.remove(nameout)
        print(f"{nameout} has been deleted.")

    #clean output in extra folders

    # Specify the directory path where the files are located
    directory_path = "/scratch/cmuhich/gamess/restart"

    # Specify the pattern of files you want to delete using wildcard characters
    pattern = name + ".*"  # Example pattern, you can modify it as needed

    # Combine the directory path and pattern to form the full pattern
    full_pattern = os.path.join(directory_path, pattern)

    # Get a list of file paths that match the full pattern
    file_paths = glob.glob(full_pattern)

    # Check if there are any files matching the pattern
    if file_paths:
        # Iterate through the list of file paths
        for file_path in file_paths:
            # Delete each file
            os.remove(file_path)
            print(f"{file_path} has been deleted.")
    else:
        print(f"No files matching the pattern {pattern} were found in {directory_path}.")

    directory_path = "/home/cmuhich/gamess_scr"

    # Combine the directory path and pattern to form the full pattern
    full_pattern = os.path.join(directory_path, pattern)

    # Get a list of file paths that match the full pattern
    file_paths = glob.glob(full_pattern)

    # Check if there are any files matching the pattern
    if file_paths:
        # Iterate through the list of file paths
        for file_path in file_paths:
            # Delete each file
            os.remove(file_path)
            print(f"{file_path} has been deleted.")
    else:
        print(f"No files matching the pattern {pattern} were found in {directory_path}.")


def get_energy(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    energy_pattern = re.compile(r"TOTAL ENERGY =\s+([-+]?\d*\.\d+|\d+)")
    energies = []
    for line in lines:
        match = energy_pattern.search(line)
        if match:
            energy = float(match.group(1))
            energies.append(energy)
    
    
    if energies:
        print("Final Energies:")
        for i, energy in enumerate(energies, start=1):
            print(f"Run {i}: {energy} Hartree")
    else:
        print("No energy information found in the output file.")
    
    
    return energies

def get_energy_at_a_temp(e,h_correction):
    conversion_factor = 2625.5002
    energy_kj = e * conversion_factor
    energy=energy_kj+h_correction
    return energy


Converted value: 4


In [2]:
# pull up GUI for design
atoms = molecule("C6H6")
viewer = WeasWidget()
viewer.from_ase(atoms)
viewer

#atom manipulation:
# click atom or group (using the shift key)
# to move press "g" then do the motion
#to rotate press "r" then do the motion
# to duplicate press "d" 
# to delete press the delete key

WeasWidget(atomScales=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], atoms={'species': {'C': ['C', 6], 'H': ['H', 1]}, …

In [3]:
# store information
atoms=viewer.to_ase()

In [4]:
#make calcuation choices
calc = GAMESSUS(contrl=dict(scftyp='rhf', dfttyp='blyp', runtyp='optimize'),
                basis=dict(gbasis='n31', ngauss=6, ndfunc=1, npfunc=1),
               statpt=dict(nstep=20))

In [5]:
#select name and run job

name='benzene'
namein= name + '.inp'
nameout= name + '.out'


atoms.set_calculator(calc)
calc.write_input( atoms)
! mv gamess_us.inp $namein

clean_for_new_job(name)

! ~/bin/rungms_basic $namein 00 $ncor > $nameout

benzene.out has been deleted.
No files matching the pattern benzene.* were found in /scratch/cmuhich/gamess/restart.
/home/cmuhich/gamess_scr/benzene.dat has been deleted.
cp ./benzene.inp /scratch/cmuhich/gamess_scr/benzene.F05
unset echo
/home/onwokonk/gamess_basic/ddikick.x /home/onwokonk/gamess_basic/gamess.00.x benzene -ddi 1 4 c008.sol.rc.asu.edu:cpus=4 -scr /scratch/cmuhich/gamess_scr
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
unset echo
ls: No match.
ls: No match.
ls: No match.


In [6]:
energies=get_energy(nameout)


Final Energies:
Run 1: -232.1409615324 Hartree
Run 2: -232.1417810194 Hartree


In [7]:
! grep NSERCH $nameout

 BEGINNING GEOMETRY SEARCH POINT NSERCH=   0 ...
          NSERCH=  0     ENERGY=    -232.1409615
 NSERCH:   0  E=     -232.1409615324  GRAD. MAX=  0.0061279  R.M.S.=  0.0031053
 BEGINNING GEOMETRY SEARCH POINT NSERCH=   1 ...
          NSERCH=  1     ENERGY=    -232.1416309
 NSERCH:   1  E=     -232.1416309379  GRAD. MAX=  0.0031385  R.M.S.=  0.0014151
 BEGINNING GEOMETRY SEARCH POINT NSERCH=   2 ...
          NSERCH=  2     ENERGY=    -232.1417808
 NSERCH:   2  E=     -232.1417807738  GRAD. MAX=  0.0002069  R.M.S.=  0.0000979
 BEGINNING GEOMETRY SEARCH POINT NSERCH=   3 ...
          NSERCH=  3     ENERGY=    -232.1417810
 NSERCH:   3  E=     -232.1417810194  GRAD. MAX=  0.0000759  R.M.S.=  0.0000226


In [8]:
atom = ase.io.read(nameout)
viewer.from_ase(atom)
viewer

WeasWidget(atomScales=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], atoms={'species': {'C': ['C', 6], 'H': ['H', 1]}, …

In [ ]:
atoms=viewer.to_ase()

In [24]:
atom = ase.io.iread(nameout)


num_positions_sets = len(atoms)
print("Number of position sets:", num_positions_sets)



for i, frame in enumerate(atom, start=1):
    print(f"Position set {i}:")
    print(frame.positions)
    print()

atom = ase.io.iread(nameout)


num_positions_sets = len(atoms)
print("Number of position sets:", num_positions_sets)



for i, frame in enumerate(atom, start=1):
    print(f"Position set {i}:")
    print(frame.positions)
    print()
    

#atom.get_positions()

Number of position sets: 12
Position set 1:
[[ 0.          1.3952479   0.        ]
 [ 1.20831991  0.69762395  0.        ]
 [ 1.20831991 -0.69762395  0.        ]
 [ 0.         -1.3952479   0.        ]
 [-1.20831991 -0.69762395  0.        ]
 [-1.20831991  0.69762395  0.        ]
 [ 0.          2.48235982  0.        ]
 [ 2.14978684  1.24117991  0.        ]
 [ 2.14978684 -1.24117991  0.        ]
 [ 0.         -2.48235982  0.        ]
 [-2.14978684 -1.24117991  0.        ]
 [-2.14978684  1.24117991  0.        ]]

Number of position sets: 12
Position set 1:
[[ 0.          1.3952479   0.        ]
 [ 1.20831991  0.69762395  0.        ]
 [ 1.20831991 -0.69762395  0.        ]
 [ 0.         -1.3952479   0.        ]
 [-1.20831991 -0.69762395  0.        ]
 [-1.20831991  0.69762395  0.        ]
 [ 0.          2.48235982  0.        ]
 [ 2.14978684  1.24117991  0.        ]
 [ 2.14978684 -1.24117991  0.        ]
 [ 0.         -2.48235982  0.        ]
 [-2.14978684 -1.24117991  0.        ]
 [-2.14978684

In [10]:
#vibrational spectra


#make calcuation choices
calc = GAMESSUS(contrl=dict(scftyp='rhf', dfttyp='blyp', runtyp='Hessian'),
                basis=dict(gbasis='n31', ngauss=6, ndfunc=1, npfunc=1),
                force=dict( METHOD='SEMINUM') )


#select name and run job

name='hessian'
namein= name + '.inp'
nameout= name + '.out'


atoms.set_calculator(calc)
calc.write_input( atoms)
! mv gamess_us.inp $namein

clean_for_new_job(name)

! ~/bin/rungms_basic $namein 00 $ncor > $nameout


hessian.out has been deleted.
No files matching the pattern hessian.* were found in /scratch/cmuhich/gamess/restart.
/home/cmuhich/gamess_scr/hessian.dat has been deleted.
cp ./hessian.inp /scratch/cmuhich/gamess_scr/hessian.F05
unset echo
/home/onwokonk/gamess_basic/ddikick.x /home/onwokonk/gamess_basic/gamess.00.x hessian -ddi 1 4 c008.sol.rc.asu.edu:cpus=4 -scr /scratch/cmuhich/gamess_scr
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
unset echo
ls: No match.
ls: No match.
ls: No match.


In [12]:
#this will tell you the frequencies in cm-1
! grep 'FREQUENCY' $nameout

!grep 'IR INTENSITY' $nameout

       FREQUENCY:        96.86       96.36       96.36        0.03        0.02  
       FREQUENCY:         0.01      381.27      381.72      602.28      603.82  
       FREQUENCY:       638.85      690.00      813.31      814.09      918.12  
       FREQUENCY:       918.21      960.80      993.78     1016.32     1040.67  
       FREQUENCY:      1047.36     1147.97     1170.09     1174.78     1335.41  
       FREQUENCY:      1383.01     1488.73     1493.97     1625.68     1629.82  
       FREQUENCY:      3143.90     3153.10     3153.96     3168.89     3169.70  
       FREQUENCY:      3180.47  
    IR INTENSITY:      0.00000     0.00000     0.00000     0.00000     0.00000
    IR INTENSITY:      0.00000     0.00000     0.00000     0.00000     0.00000
    IR INTENSITY:      1.47712     0.00000     0.00000     0.00000     0.00000
    IR INTENSITY:      0.00000     0.00000     0.00002     0.00000     0.07726
    IR INTENSITY:      0.07643     0.00006     0.00000     0.00000     0.00000
    I

In [16]:
! grep -C 10 'E         H         G         CV        CP        S' $nameout

        0.098508 HARTREE/MOLECULE    21620.050337 CM**-1/MOLECULE 
       61.814888 KCAL/MOL              258.633491 KJ/MOL

               Q               LN Q
 ELEC.     1.00000E+00       0.000000
 TRANS.    2.71011E+07      17.115087
 ROT.      9.02677E+04      11.410535
 VIB.      1.93327E+00       0.659213
 TOT.      4.72947E+12      29.184835

              E         H         G         CV        CP        S
           KJ/MOL    KJ/MOL    KJ/MOL   J/MOL-K   J/MOL-K   J/MOL-K
 ELEC.      0.000     0.000     0.000     0.000     0.000     0.000
 TRANS.     3.718     6.197   -42.427    12.472    20.786   163.088
 ROT.       3.718     3.718   -28.286    12.472    12.472   107.343
 VIB.     263.222   263.222   256.999    50.578    50.578    20.869
 TOTAL    270.658   273.137   186.286    75.521    83.836   291.301
 VIB. THERMAL CORRECTION E(T)-E(0) = H(T)-H(0) =      4588.077 J/MOL

              E         H         G         CV        CP        S
         KCAL/MOL  KCAL/MOL  KCAL/MOL 

In [23]:
energies=get_energy(nameout)

#type in the energy from the above table
correction=273.137

energy_at_temp=get_energy_at_a_temp(energies[0],correction)
print('the energy in kJ/mol is:', energy_at_temp)

Final Energies:
Run 1: -232.1409615324 Hartree
the energy in kJ/mol is: -609213.0039315085
